# TFace

> Face Image Quality model from [SDD-FIQA](https://github.com/Tencent/TFace).

In [ ]:
#| default_exp tface

In [ ]:
#| hide

from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import numpy as np
import torch

from tface.utils import network, preprocess_tf_img

In [ ]:
#| export 

def get_network():

    if torch.cuda.is_available():
        device = 'cuda:0'
        net = network(device=device)
    elif not torch.cuda.is_available():
        device = 'cpu'
        net = network( device=device)        
    else:
        raise ValueError(f'Value {torch.cuda.is_available()} is not correct')

    return net


In [ ]:
#| export

def compute_tf_quality(aligned_img:np.array, network)->float:
    
    net = network

    if torch.cuda.is_available():
        
        input_data = preprocess_tf_img(aligned_img).to('cuda:0')
        pred_score = net(input_data).data.cuda().numpy().squeeze()     
        
    elif not torch.cuda.is_available():
        
        input_data = preprocess_tf_img(aligned_img)
        pred_score = net(input_data).data.cpu().numpy().squeeze()
           
    else:
        raise ValueError(f'Value {torch.cuda.is_available()} is not correct')

    return pred_score



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()